In [1]:
%pip install seaborn

%matplotlib inline

%pip install requests 

%pip install openpyxl

%pip install xlrd

%pip install regex

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import warnings
warnings.filterwarnings('ignore')

import regex as re

import pylab as plt
import seaborn as sns

Se cargan las bases de datos y se revisan para comenzar a limpiarlas.

In [3]:
actor = pd.read_csv('../data/actor.csv')

In [4]:
category = pd.read_csv('../data/category.csv')

In [5]:
film = pd.read_csv('../data/film.csv')

In [6]:
inventory = pd.read_csv('../data/inventory.csv')

In [7]:
language = pd.read_csv('../data/language.csv')

In [8]:
old_HDD = pd.read_csv('../data/old_HDD.csv')

In [9]:
rental = pd.read_csv('../data/rental.csv')

Eliminamos duplicados si los hubiese en cada una de las df

In [10]:
actor.drop_duplicates(inplace=True) 
film.drop_duplicates(inplace=True) 
inventory.drop_duplicates(inplace=True) 
language.drop_duplicates(inplace=True) 
old_HDD.drop_duplicates(inplace=True) 
rental.drop_duplicates(inplace=True) 

Revisamos columnas con nulos que no aporten información

In [11]:
nancolsactor = actor.isnull().sum()

nancolsactor

actor_id       0
first_name     0
last_name      0
last_update    0
dtype: int64

In [12]:
nancolsfilm = film.isnull().sum()

nancolsfilm

film_id                    0
title                      0
description                0
release_year               0
language_id                0
original_language_id    1000
rental_duration            0
rental_rate                0
length                     0
replacement_cost           0
rating                     0
special_features           0
last_update                0
dtype: int64

In [13]:
film.shape

(1000, 13)

In [14]:
nancolsinventory = inventory.isnull().sum()

nancolsinventory

inventory_id    0
film_id         0
store_id        0
last_update     0
dtype: int64

In [15]:
nancolslanguage = language.isnull().sum()

nancolslanguage

language_id    0
name           0
last_update    0
dtype: int64

In [16]:
nancolsold = old_HDD.isnull().sum()

nancolsold

first_name      0
last_name       0
title           0
release_year    0
category_id     0
dtype: int64

In [17]:
nancolsrental = rental.isnull().sum()

nancolsrental

rental_id       0
rental_date     0
inventory_id    0
customer_id     0
return_date     0
staff_id        0
last_update     0
dtype: int64

No hay valores nulos en las tablas salvo en la columna 'original_language_id' de film. Se elimina esta columna tras comprobar que el 100% son nulos (coincide con las dimensiones de la tabla)

In [18]:
film.drop(['original_language_id'], axis=1, inplace=True)

film.columns

Index(['film_id', 'title', 'description', 'release_year', 'language_id',
       'rental_duration', 'rental_rate', 'length', 'replacement_cost',
       'rating', 'special_features', 'last_update'],
      dtype='object')

Se observa que 'name' en language y 'name en 'category' son diferentes y pueden llevar a confusión, por lo que se cambia el nombre de cada ID.

In [19]:
language.rename(columns={'name': 'name_lang'}, inplace=True)

category.rename(columns={'name': 'name_cat'}, inplace=True)

2) Se eliminan las columnas que no aportan valor estadístico o de estudio de mercado para liberar espacio. Ejemplos: 'store_id' en inventory (identificador del lugar de almacenamiento); 'last_update' en todas las tablas salvo en old_HDD (última actualización de los datos); 'rental_date' y 'return_date' de rental no aporta información mas que cuándo se alquilan las películas y se devuelven, por lo que no aporta demasiada información.

In [20]:
inventory.drop(['last_update'], axis=1, inplace=True)
rental.drop(['last_update'], axis=1, inplace=True)
category.drop(['last_update'], axis=1, inplace=True)
film.drop(['last_update'], axis=1, inplace=True)
actor.drop(['last_update'], axis=1, inplace=True)
language.drop(['last_update'], axis=1, inplace=True)

Iremos analizando de una en una las diferentes dataframe para revisar su contenido y analizar sus datos y si el contenido es relevante o no.

### Tabla actor 

In [21]:
actor.head()

,actor_id,first_name,last_name
0,1,PENELOPE,GUINESS
1,2,NICK,WAHLBERG
2,3,ED,CHASE
3,4,JENNIFER,DAVIS
4,5,JOHNNY,LOLLOBRIGIDA


Se van a unir nombre con apellido para dejar asignados los nombres y apellidos de la tabla actor para poder compararlo con los nombres y apellidos de old_HDD.

In [22]:
actor['name_act']=actor.first_name + ' ' + actor.last_name

In [23]:
actor.head()

,actor_id,first_name,last_name,name_act
0,1,PENELOPE,GUINESS,PENELOPE GUINESS
1,2,NICK,WAHLBERG,NICK WAHLBERG
2,3,ED,CHASE,ED CHASE
3,4,JENNIFER,DAVIS,JENNIFER DAVIS
4,5,JOHNNY,LOLLOBRIGIDA,JOHNNY LOLLOBRIGIDA


In [24]:
actor.name_act.value_counts()

SUSAN DAVIS             2
EWAN GOODING            1
DARYL CRAWFORD          1
GRETA KEITEL            1
JANE JACKMAN            1
                       ..
MICHELLE MCCONAUGHEY    1
ADAM GRANT              1
SEAN WILLIAMS           1
GARY PENN               1
THORA TEMPLE            1
Name: name_act, Length: 199, dtype: int64

Se observa que hay un valor repetido que puede suponer la entrada duplicada de un dato, o que un actor/actriz tenga el mismo nombre. Se investiga.

In [25]:
actor.loc[actor['name_act'] == 'SUSAN DAVIS']

,actor_id,first_name,last_name,name_act
100,101,SUSAN,DAVIS,SUSAN DAVIS
109,110,SUSAN,DAVIS,SUSAN DAVIS


Dado que el actor_id no está relacionado con otra columna para poder comprobar si es correcto o no el id 101 o 110, se va a eliminar uno de ellos por estar repetido. Se interpreta que es un valor repetido y que no hay dos 'Susan Davis'.

In [26]:
rowrep= actor[actor['actor_id'] == 110]

rowrep= list(rowrep.index)

actor.drop(rowrep, axis=0, inplace=True)

In [27]:
actor.loc[actor['actor_id'] == 110]

,actor_id,first_name,last_name,name_act


### Tabla old_HDD

Se añade la columna name_act en el que aparece el nombre y apellido del actor/actriz.

In [28]:
old_HDD.head()

,first_name,last_name,title,release_year,category_id
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,2006,6
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2006,2
2,PENELOPE,GUINESS,ANGELS LIFE,2006,13
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,2006,10
4,PENELOPE,GUINESS,CHEAPER CLYDE,2006,14


In [29]:
old_HDD['name_act']=old_HDD.first_name + ' ' + old_HDD.last_name

In [30]:
old_HDD.head()

,first_name,last_name,title,release_year,category_id,name_act
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,2006,6,PENELOPE GUINESS
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2006,2,PENELOPE GUINESS
2,PENELOPE,GUINESS,ANGELS LIFE,2006,13,PENELOPE GUINESS
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,2006,10,PENELOPE GUINESS
4,PENELOPE,GUINESS,CHEAPER CLYDE,2006,14,PENELOPE GUINESS


In [31]:
old_HDD=old_HDD.iloc[:, [0,1,5,2,3,4]]

old_HDD.head()

,first_name,last_name,name_act,title,release_year,category_id
0,PENELOPE,GUINESS,PENELOPE GUINESS,ACADEMY DINOSAUR,2006,6
1,PENELOPE,GUINESS,PENELOPE GUINESS,ANACONDA CONFESSIONS,2006,2
2,PENELOPE,GUINESS,PENELOPE GUINESS,ANGELS LIFE,2006,13
3,PENELOPE,GUINESS,PENELOPE GUINESS,BULWORTH COMMANDMENTS,2006,10
4,PENELOPE,GUINESS,PENELOPE GUINESS,CHEAPER CLYDE,2006,14


No hay mucho más que trabajar en estas tablas. Se observa que habrá datos que faltarán al unificar las tablas en la base de datos ya que no todos los títulos de film están en old_HDD cuyos actores sólo están vinculados a esta tabla. Lo mismo ocurre con los datos de 'inventory_id' en rental donde se tienen más registros que no se contemplan en 'inventory_id' de inventory por lo que también aparecerán datos nulos. 

In [74]:
old_HDD.title.nunique()

614

In [75]:
film.title.nunique()

1000

In [85]:
inventory['inventory_id'].value_counts()

1       1
672     1
659     1
660     1
661     1
       ..
339     1
340     1
341     1
342     1
1000    1
Name: inventory_id, Length: 1000, dtype: int64

In [84]:
rental['inventory_id'].value_counts()

367     1
1769    1
1414    1
2247    1
2968    1
       ..
946     1
4168    1
4019    1
3301    1
1498    1
Name: inventory_id, Length: 1000, dtype: int64

Se exportan las nuevas tablas para trabajar en SQL.

In [88]:
actor.to_csv('../data/actor2.csv', index=False)
category.to_csv('../data/category2.csv', index=False)
film.to_csv('../data/film2.csv', index=False)
inventory.to_csv('../data/inventory2.csv', index=False)
language.to_csv('../data/language2.csv', index=False)
old_HDD.to_csv('../data/old_HDD2.csv', index=False)
rental.to_csv('../data/rental2.csv', index=False)